In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
# di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))


import hail as hl
hl.init(tmp_dir='/net/scratch/people/plggosborcz')

Running on Apache Spark version 2.4.3
SparkUI available at http://p1110.prometheus:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /net/archive/groups/plggneuromol/sportsmen-wgs/hail-20210329-1330-0.2.62-84fa81b9ea3d.log


In [3]:
pwd

'/net/archive/groups/plggneuromol/sportsmen-wgs'

In [2]:
ls

                                            job-log--46654818.txt
analysis/                                   job-log--47029885.txt
bam/                                        joint-vcf/
fastq/                                      joint-vcf-preprocessing.ipynb
filter-gene-list-visualise-pca.ipynb        mts/
gvcf/                                       python-notebook.slurm*
hail-20210211-1520-0.2.30-2ae07d872f43.log  sportsmen-gene-list.csv
hail-20210212-0946-0.2.30-2ae07d872f43.log  vcf/
hail-20210212-1427-0.2.30-2ae07d872f43.log  vcf-joint-gts/


In [6]:
import pandas

AttributeError: module 'pandas' has no attribute 'tseries'

In [2]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()

import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain
import statistics as stat

from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

Loading BokehJS ...

In [4]:
# Install a pip package in the current Jupyter kernel :)
import sys
!{sys.executable} -m pip install openpyxl

     |████████████████████████████████| 242 kB 1.1 MB/s eta 0:00:01
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-py3-none-any.whl size=8917 sha256=755f41b2e8f67fc7e763daf90bb54e9b4d4d0be18c519655b68013e5ce4fb9e7
  Stored in directory: /root/.cache/pip/wheels/e2/bd/55/048b4fd505716c4c298f42ee02dffd9496bb6d212b266c7f31
Successfully built et-xmlfile


In [3]:
mt = hl.read_matrix_table('data/mts/sportsmen-small.mt/')

FatalError: HailException: MatrixTable and Table files are directories; path 'data/mts/sportsmen-small.mt/' is not a directory

Java stack trace:
is.hail.utils.HailException: MatrixTable and Table files are directories; path 'data/mts/sportsmen-small.mt/' is not a directory
	at is.hail.utils.ErrorHandling$class.fatal(ErrorHandling.scala:11)
	at is.hail.utils.package$.fatal(package.scala:77)
	at is.hail.expr.ir.RelationalSpec$.readMetadata(AbstractMatrixTableSpec.scala:32)
	at is.hail.expr.ir.RelationalSpec$.readReferences(AbstractMatrixTableSpec.scala:66)
	at is.hail.variant.ReferenceGenome$.fromHailDataset(ReferenceGenome.scala:596)
	at is.hail.variant.ReferenceGenome.fromHailDataset(ReferenceGenome.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java

In [4]:
mt.count()

NameError: name 'mt' is not defined

### Visualise PCA results:

In [5]:
p = hl.plot.scatter(mt.scores[0],
                    mt.scores[1],
                    label=(mt.s),
                    title='PCA', xlabel='PC1', ylabel='PC2')

show(p)

In [6]:
pc_rel = hl.pc_relate(mt.GT, 0.001, k=2, statistics='kin')
pairs = pc_rel.filter(pc_rel['kin'] > 0.125)

2020-11-30 19:31:29 Hail: INFO: hwe_normalized_pca: running PCA using 6206 variants.
2020-11-30 19:31:30 Hail: INFO: pca: running PCA with 2 components...
2020-11-30 19:32:27 Hail: INFO: Wrote all 2 blocks of 6280 x 102 matrix with block size 4096.


In [8]:
related_samples_to_remove = hl.maximal_independent_set(pairs.i, pairs.j, keep=False)

2020-11-30 19:32:30 Hail: INFO: wrote matrix with 3 rows and 6280 columns as 2 blocks of size 4096 to /tmp/pcrelate-write-read-4VHAUCBBDGRH29boHoMTOH.bm
2020-11-30 19:32:30 Hail: INFO: wrote matrix with 6280 rows and 102 columns as 2 blocks of size 4096 to /tmp/pcrelate-write-read-oTHlrUwQKHUgU1ulpYeD97.bm
2020-11-30 19:32:30 Hail: INFO: wrote matrix with 102 rows and 102 columns as 1 block of size 4096 to /tmp/pcrelate-write-read-L59jqGLhyLS8iz5OZv9SZf.bm
2020-11-30 19:32:31 Hail: INFO: wrote matrix with 102 rows and 102 columns as 1 block of size 4096 to /tmp/pcrelate-write-read-8qzBfy1HyQd5xeg7Lop5dy.bm
2020-11-30 19:32:31 Hail: INFO: wrote matrix with 102 rows and 102 columns as 1 block of size 4096 to /tmp/pcrelate-write-read-Spwnjn93Pvk4x21kBeRCuD.bm
2020-11-30 19:32:31 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-11-30 19:32:31 Hail: INFO: wrote table with 6 rows in 1 partition to /tmp/bSrkCHjv1urtfymR8WN3iy
    Total size: 114.00 B
    * Rows: 103.00 B
    * 

In [9]:
    
related_samples_to_remove = related_samples_to_remove.annotate(s = related_samples_to_remove.node.s)
related_samples_to_remove = related_samples_to_remove.key_by('s')
    
mt = mt.key_cols_by()

In [10]:
mt = mt.filter_cols(hl.is_defined(related_samples_to_remove[mt.s]), keep=False)
mt = mt.key_cols_by(mt.s)

2020-11-30 19:32:31 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


In [11]:
mt.count()

2020-11-30 19:32:32 Hail: INFO: Coerced sorted dataset
2020-11-30 19:32:32 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-11-30 19:32:32 Hail: INFO: Coerced sorted dataset


(6280, 96)

In [12]:
related_samples_to_remove.show()

2020-11-30 19:32:33 Hail: INFO: Ordering unsorted dataset with network shuffle


,
node,
s,s
str,str
"""B156""","""B156"""
"""B382""","""B382"""
"""B427""","""B427"""
"""B449""","""B449"""
"""B478""","""B478"""
"""B507""","""B507"""


### Gather gene lists:

HPO terms representing fatigue and abnormal cardiac tests, mouse genes from these two papers:

https://pubmed.ncbi.nlm.nih.gov/30967789/
https://pubmed.ncbi.nlm.nih.gov/29910734/

+ 111 genes enriched in skeletal muscle
https://www.proteinatlas.org/search/tissue_category_rna%3Askeletal+muscle%3BTissue+enriched+AND+sort_by%3Atissue+specific+score+AND+show_columns%3Agroupenriched

In [25]:
genes = pd.read_csv('/hail/data/data/sportsmen-gene-list.csv', header=None)

In [29]:
genes = list(genes[0])

In [31]:
mt_small = hl.read_matrix_table('data/mts/sportsmen-small.mt/')

In [43]:
cp = hl.plot.histogram(mt.DANN[0], bins=100, title='DANN Histogram', legend='DANN scores')
show(p)

In [152]:
mt = hl.read_matrix_table('data/mts/sportsmen.mt/')

In [153]:
voi = mt.filter_rows(hl.any(lambda x: hl.literal(genes).contains(x), mt.nearest_genes_20kb))

In [75]:
voi.count()

(140337, 102)

In [154]:
voi = voi.filter_rows(voi.DANN[0] > 0.95)

In [77]:
voi.count()

(1932, 102)

In [78]:
cp = hl.plot.histogram(voi.gnomad_v3_1.cadd.phred, title='CADD Histogram', legend='CADD scores')
show(cp)

In [155]:
voi = voi.filter_rows(voi.gnomad_v3_1.cadd.phred > 30)

In [124]:
voi.count()

(58, 102)

In [156]:
voi_short = voi.filter_rows(hl.any(lambda x: hl.literal(genes).contains(x), voi.within_gene))

In [131]:
voi_short.count()

(41, 102)

### Export interesting variants as csv

In [157]:
voi_short = voi_short.annotate_rows(
    samples_het = hl.agg.filter(voi_short.GT.is_het(), hl.agg.collect(voi_short.s)),
    samples_hom_non_ref = hl.agg.filter(voi_short.GT.is_hom_var(), hl.agg.collect(voi_short.s)))

voi_short = voi_short.transmute_rows(
    depth = voi_short.info.DP,
    gnomAD_v3_1_allele_frequency = voi_short.gnomad_v3_1.freq[0].AF,
    gnomAD_v3_1_homozygote_count = voi_short.gnomad_v3_1.freq[0].homozygote_count,
    rsid = voi_short.gnomad_v3_1.rsid,
    predicted_most_severe_consequence = voi_short.gnomad_v3_1.vep.most_severe_consequence,
    predicted_loss_of_function = voi_short.gnomad_v3_1.vep.transcript_consequences.lof,
    revel_score = voi_short.gnomad_v3_1.revel.revel_score,
    primate_ai_score = voi_short.gnomad_v3_1.primate_ai.primate_ai_score,
    gene = voi_short.within_gene,
    hpo = voi_short.hpo,
    clinvar_significance = voi_short.clinvar_variant_summary.ClinicalSignificance,
    samples_het = voi_short.samples_het,
    samples_hom_non_ref = voi_short.samples_hom_non_ref,
    dann = voi_short.DANN.score,
    cadd = voi_short.gnomad_v3_1.cadd.phred
)

voi_short = voi_short.drop(voi_short.clinvar_gene_summary)

to_export = voi_short.rows()

to_export = to_export.to_pandas()

In [174]:
to_export.columns

Index(['locus.contig', 'locus.position', 'alleles', 'rsid', 'qual', 'hpo',
       'nearest_genes_20kb', 'samples_het', 'samples_hom_non_ref', 'depth',
       'gnomAD_v3_1_allele_frequency', 'gnomAD_v3_1_homozygote_count',
       'predicted_most_severe_consequence', 'revel_score', 'primate_ai_score',
       'gene', 'clinvar_significance', 'dann', 'cadd'],
      dtype='object')

In [175]:
to_export

,locus.contig,locus.position,alleles,rsid,qual,hpo,nearest_genes_20kb,samples_het,samples_hom_non_ref,depth,gnomAD_v3_1_allele_frequency,gnomAD_v3_1_homozygote_count,predicted_most_severe_consequence,revel_score,primate_ai_score,gene,clinvar_significance,dann,cadd
0,chr1,26067352,"[G, A]",rs140523053,673.56,[None],"[AL391650.2, TRIM63]","[B479, B537]",[],3389,0.001268,0,missense_variant,0.557,0.758671,[TRIM63],"[Uncertain significance, Pathogenic, Pathogenic]",0.999289,31.0
1,chr1,63623761,"[G, A]",None,3351.72,"[[HP:0004322:Short stature, HP:0001644:Dilated...",[PGM1],"[B382, B474, B508, B522, B523, B528]",[],3866,0.010201,24,splice_donor_variant,NaN,NaN,[PGM1],"[Uncertain significance, Pathogenic, Pathogeni...",0.995049,33.0
2,chr1,74370392,"[G, C]",rs145260115,497.77,"[[HP:0001692:Atrial arrhythmia, HP:0011712:Rig...","[AC093158.1, FPGT-TNNI3K, TNNI3K]",[B456],[],3600,0.001087,0,missense_variant,0.628,NaN,"[AC093158.1, FPGT-TNNI3K, TNNI3K]","[Uncertain significance, Pathogenic, Uncertain...",0.970179,34.0
3,chr1,113829592,"[C, G]",rs56048322,2311.04,"[[HP:0000083:Renal insufficiency, HP:0002960:A...","[AL137856.1, PTPN22, RSBN1]","[B444, B457, B493, B504, B531]",[],3746,0.005758,9,missense_variant,0.186,0.442757,"[AL137856.1, PTPN22]","[Pathogenic, Pathogenic, Pathogenic, Pathogeni...",0.998572,35.0
4,chr1,114693436,"[G, A]",None,15771.42,"[[HP:0003690:Limb muscle weakness, HP:0003394:...","[AMPD1, NRAS, RN7SL432P]","[B433, B435, B437, B454, B455, B456, B462, B46...","[B444, B464]",3400,0.084976,761,stop_gained,NaN,NaN,[AMPD1],"[Uncertain significance, Pathogenic, Pathogeni...",0.996609,38.0
5,chr2,32250936,"[G, A]",rs199475953,228.77,"[[HP:0000509:Conjunctivitis, HP:0001025:Urtica...",[NLRC4],[B462],[],3596,0.000151,0,stop_gained,NaN,NaN,[NLRC4],"[Uncertain significance, Pathogenic, Uncertain...",0.996254,31.0
6,chr2,71535283,"[G, A]",rs61740288,1821.02,"[[HP:0008954:Intrinsic hand muscle atrophy, HP...",[DYSF],"[B446, B465, B81]",[B444],2828,0.017176,30,missense_variant,0.405,0.580331,[DYSF],"[Uncertain significance, Pathogenic, Uncertain...",0.999090,33.0
7,chr2,71681053,"[G, A]",rs115407852,977.56,"[[HP:0008954:Intrinsic hand muscle atrophy, HP...",[DYSF],"[B431, B443]",[],3608,0.000223,0,missense_variant,0.734,0.498260,[DYSF],"[Uncertain significance, Pathogenic, Uncertain...",0.999552,31.0
8,chr2,127286874,"[G, A]",rs1428334772,207.28,"[[HP:0000988:Skin rash, HP:0004322:Short statu...","[AC068282.1, ERCC3, MAP3K2]",[B539],[],3371,0.000013,0,missense_variant,0.602,0.899358,[ERCC3],"[Pathogenic, Pathogenic, Pathogenic, Pathogeni...",0.999228,32.0
9,chr2,151501423,"[G, A]",rs549794342,694.56,"[[HP:0002068:Neuromuscular dysphagia, HP:00026...","[NEB, RIF1]","[B466, B489]",[],2989,0.000303,0,stop_gained,0.200,NaN,"[NEB, RIF1]","[Uncertain significance, Pathogenic, Pathogeni...",0.993420,51.0


In [173]:
to_export = to_export.drop(['a_index', 'was_split', ], axis=1)

In [176]:
to_export.to_csv('filtered_variants_sportsmen')

### Export to vcf

In [178]:
mt = hl.read_matrix_table('data/mts/sportsmen.mt/')

In [181]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        DP: int32, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: float64, 
        NDA: int32, 
        QD: float64, 
        RAW_MQandDP: array<int32>, 
        ReadPosRankSum: float64, 
        SOR: float64
    }
    'a_index': int32
    'was_split': bool
    'gnomad_v3_1': struct {
        freq: array<struct {
            AC: int32, 
            AF: float64, 
            AN: int32, 
            homoz

In [183]:
mt = mt.annotate_rows(info = hl.struct(AC = mt.info.AC, 
                                       AF = mt.info.AF,
                                       AN = mt.info.AN,
                                       gnomad_AF = mt.gnomad_v3_1.freq[0].AF, 
                                       gnomad_homozygote_count = mt.gnomad_v3_1.freq[0].homozygote_count,
                                       highest_consequence = mt.gnomad_v3_1.vep.most_severe_consequence,
                                       pholyphen_prediction = mt.gnomad_v3_1.vep.transcript_consequences.polyphen_prediction, 
                                       polyphen_score = mt.gnomad_v3_1.vep.transcript_consequences.polyphen_score, 
                                       sift_prediction = mt.gnomad_v3_1.vep.transcript_consequences.sift_prediction, 
                                       sift_score = mt.gnomad_v3_1.vep.transcript_consequences.sift_score,
                                       transcript_id = mt.gnomad_v3_1.vep.transcript_consequences.transcript_id, 
                                       variant_class = mt.gnomad_v3_1.vep.variant_class,
                                       gene_symbol = mt.within_gene, 
                                       hpo = hl.flatten(mt.hpo),
                                       rsid = mt.gnomad_v3_1.rsid,
                                       revel_score = mt.gnomad_v3_1.revel.revel_score,
                                       primate_ai_score = mt.gnomad_v3_1.primate_ai.primate_ai_score,
                                       clinvar_significance = mt.clinvar_variant_summary.ClinicalSignificance,
                                       dann = mt.DANN.score,
                                       cadd = mt.gnomad_v3_1.cadd.phred))

In [190]:
hl.export_vcf(mt, '/hail/data/data/sportsmen.vcf.bgz')

2020-12-01 14:19:53 Hail: WARN: export_vcf: ignored the following fields:
    'a_index' (row)
    'was_split' (row)
    'gnomad_v3_1' (row)
    'within_gene' (row)
    'hpo' (row)
    'nearest_genes_20kb' (row)
    'DANN' (row)
    'clinvar_gene_summary' (row)
    'clinvar_variant_summary' (row)
2020-12-01 14:23:06 Hail: INFO: merging 123 files totalling 4.4G...
2020-12-01 14:23:12 Hail: INFO: while writing:
    /hail/data/data/sportsmen.vcf.bgz
  merge time: 5.547s
